# Breast Cancer: (KNN, LogisticRegression)

## Librerias

In [4]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import ParameterGrid
import numpy as np
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, train_test_split, StratifiedKFold
from sklearn.datasets import load_breast_cancer

## Cargar los Datos

In [8]:
cancer = load_breast_cancer()
X, y = cancer.data, cancer.target
XTrain, XTest, yTrain, yTest = train_test_split(X, y, test_size=0.2, random_state=42)

# Metrica usada RECALL

$$
\text{Recall} = \frac{\text{Verdaderos Positivos (VP)}}{\text{Verdaderos Positivos (VP)} + \text{Falsos Negativos (FN)}}
$$
El recall emerge como una métrica de evaluación clave al trabajar en la identificación de condiciones médicas, tales como el cáncer de mama, primordialmente debido a las implicaciones significativas asociadas con los falsos negativos. Esta situación adquiere una dimensión crítica en el contexto del cáncer de mama, dado que la detección precoz y el diagnóstico acertado juegan roles fundamentales en mejorar las perspectivas y opciones de tratamiento para el paciente.


# Manual

In [22]:
from sklearn.metrics import recall_score

param_grid = {
    'KNN': {
        'model': KNeighborsClassifier(),
        'params': {
            'n_neighbors': [3, 5, 7, 9],
        }
    },
    'LogisticRegression': {
        'model': LogisticRegression(max_iter=10000),
        'params': {
            'C': [0.1, 1, 10, 100]
        }
    }
}

def customPipeline(XTrain, XTest, yTrain, yTest):
    results = []
    bestScore = 0
    bestEstimator = None

    for modelName, modelInfo in param_grid.items():
        model = modelInfo['model']
        params = modelInfo['params']

        scaler = StandardScaler()
        XTrainScaled = scaler.fit_transform(XTrain)
        XTestScaled = scaler.transform(XTest)

        modelInstance = model
        bestModel, bestScore, bestParams = None, 0, None

        for nNeighbor in params.get('n_neighbors', [None]):
            for algorithm in params.get('algorithm', [None]):
                if 'n_neighbors' in params:
                    modelInstance.set_params(n_neighbors=nNeighbor)
                if 'algorithm' in params:
                    modelInstance.set_params(algorithm=algorithm)

                modelInstance.fit(XTrainScaled, yTrain)

                cv = StratifiedKFold(n_splits=5)
                scores = []
                recalls = []

                for trainIdx, valIdx in cv.split(XTrainScaled, yTrain):
                    XTrainFold, XValFold = XTrainScaled[trainIdx], XTrainScaled[valIdx]
                    yTrainFold, yValFold = yTrain[trainIdx], yTrain[valIdx]

                    modelInstance.fit(XTrainFold, yTrainFold)
                    yPred = modelInstance.predict(XValFold)
                    recall = recall_score(yValFold, yPred)
                    recalls.append(recall)

                meanRecall = sum(recalls) / len(recalls)

                if meanRecall > bestScore:
                    bestScore = meanRecall
                    bestModel = modelInstance
                    bestParams = {'n_neighbors': nNeighbor}

        results.append({
            'Model': modelName,
            'Best Recall': bestScore,
            'Best Parameters': bestParams
        })

    for result in results:
        print(f"Model: {result['Model']}, Best Recall: {result['Best Recall']}, Best Parameters: {result['Best Parameters']}")

customPipeline(XTrain, XTest, yTrain, yTest)


Model: KNN, Best Recall: 0.9930429522081065, Best Parameters: {'n_neighbors': 9}
Model: LogisticRegression, Best Recall: 0.9860859044162129, Best Parameters: {'n_neighbors': None}




### K-Nearest Neighbors (KNN)
- El modelo KNN fue capaz de identificar correctamente el 99.30% de los casos positivos de cáncer de mama. Es una tasa muy alta, lo que sugiere que el modelo es muy eficaz en la detección de casos de cáncer de mama.
- Este resultado muestra que el mejor rendimiento se logró cuando el número de vecinos más cercanos fue 9. 

### Regresión Logística
- El modelo de Regresión Logística fue capaz de identificar correctamente el 98.61% de los casos positivos de cancer. Aunque es ligeramente inferior al desempeño del modelo KNN, sigue siendo una tasa de detección muy alta.

Ambos modelos muestran un alto recall, lo que es deseable en el contexto del diagnóstico del cáncer de mama, donde es crucial minimizar los falsos negativos.

#  Con metodos

In [23]:
pipelines = {
    'knn': Pipeline([
        ('scaler', StandardScaler()),
        ('knn', KNeighborsClassifier())
    ]),
    'logistic_regression': Pipeline([
        ('scaler', StandardScaler()),
        ('logistic_regression', LogisticRegression(max_iter=10000))
    ])
}

param_grid = {
    'knn': {
        'knn__n_neighbors': [3, 5, 7, 9],
    },
    'logistic_regression': {
        'logistic_regression__C': [0.1, 1, 10, 100]
    }
}

# Paso 5: Ejecutar GridSearchCV para cada modelo
for model in pipelines:
    grid_search = GridSearchCV(pipelines[model], param_grid[model], cv=StratifiedKFold(n_splits=5), scoring='recall', verbose=1)
    grid_search.fit(XTrain, yTrain)
    
    print(f"Model {model}: Best Recall {model}:, {grid_search.best_score_} Best Parameters: {grid_search.best_params_}")


Fitting 5 folds for each of 4 candidates, totalling 20 fits
Model knn: Best Recall knn:, 0.9895946763460376 Best Parameters: {'knn__n_neighbors': 5}
Fitting 5 folds for each of 4 candidates, totalling 20 fits
Model logistic_regression: Best Recall logistic_regression:, 0.9930429522081065 Best Parameters: {'logistic_regression__C': 0.1}



### K-Nearest Neighbors (KNN)
- El modelo KNN logró identificar correctamente el 98.96%  la presencia de cáncer de mama, lo que sugiere que el modelo es altamente efectivo 
- Este resultado indica que el mejor desempeño se logró cuando el número de vecinos más cercanos alcanzó el 5. 

### Regresión Logística
- El modelo de Regresión Logística identificó correctamente el 99.30% de los casos positivos en el conjunto de datos de cancer.

- Los mejores resultados se obtuvieron con un valor de `0.1` para el parámetro de regularización 'C', que controla la fuerza de la regularización en el modelo de Regresión Logística. Un valor más bajo de 'C' indica una regularización más fuerte, lo que puede ayudar a prevenir el sobreajuste al hacer que el modelo sea más simple.




#### En conclusión, comparando los resultados obtenidos manualmente utilizando métodos existentes, hay algunas diferencias. En el primer análisis, KNN muestra mejores resultados, mientras que en el segundo, la Regresión Logística alcanza el mejor recall. Sin embargo, en ambos casos, ambos modelos muestran resultados muy buenos y similares.